In [34]:
%load_ext autoreload
%autoreload 2

%pip install -r requirements.txt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Note: you may need to restart the kernel to use updated packages.


In [35]:
import sys
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
import torchvision.models as models
from torchvision import transforms
from datasets import load_dataset, concatenate_datasets

In [36]:
print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Check for CUDA support
print(f"Is CUDA available? {torch.cuda.is_available()}")

# Set the device
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using device: {device}")


PyTorch version: 2.1.0+cu118
Is MPS (Metal Performance Shader) built? False
Is MPS available? False
Is CUDA available? True
Using device: cuda


In [37]:
# pick which model to load
model_name = "densenet_cifar100" # either "resnet" or "vgg_cifar10" or "vgg_cifar100"
num_classes = 100 if "cifar100" in model_name else 10
model_path = os.path.join("models", model_name)

In [38]:
from DataLoader import CustomDataset
import numpy as np

if model_name == "resnet":
    # use the imagenette dataset
    hf_dataset = load_dataset("frgfm/imagenette", '320px')
    hf_dataset = concatenate_datasets(hf_dataset.values())
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

elif model_name == "vgg_cifar10":
    # use the cifar10 dataset
    hf_dataset = load_dataset("cifar10")
    hf_dataset = concatenate_datasets(hf_dataset.values())
    
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.507, 0.4865, 0.4409],
                             std=[0.2673, 0.2564, 0.2761])
    ])
elif "cifar100" in model_name:
    # use the cifar100 dataset
    hf_dataset = load_dataset("cifar100")
    hf_dataset = concatenate_datasets(hf_dataset.values())
    
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
    ])
    
torch_dataset = CustomDataset(hf_dataset, transform=transform)

batch_size = 32 if model_name == "resnet" else 64

test_size = 0.2
test_volume = int(test_size * len(torch_dataset))
train_volume = len(torch_dataset) - test_volume

train_dataset, test_dataset = random_split(torch_dataset, [train_volume, test_volume])
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=False, 
    num_workers=4
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4
)

In [39]:
import ModelLoader

loader = ModelLoader.ModelLoader(model_name, device, dataloader=train_dataloader)

# preview the model architecture
model = loader.load_model(num_outputs=num_classes, trained_classifiers=True)
model

Loading EarlyExit DenseNet121 model architecture...
Adding exits...
Setting model weights...


EarlyExitModel(
  (model): DenseNet(
    (conv1): Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (dense1): Sequential(
      (0): Bottleneck(
        (bn1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(24, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (1): Bottleneck(
        (bn1): BatchNorm2d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(36, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(48, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (2): Bottleneck(
        (bn1): BatchNorm2d(48, eps

## Early Exit Model Training

If you want tensorboard support, you need to run the following commands on **macOS**:

This clears your logs:
```
rm -rf runs/*
```

This shows your public IP address:
```
dig -4 TXT +short o-o.myaddr.l.google.com @ns1.google.com
```

This starts tensorboard at this address:
```
tensorboard --host 0.0.0.0 --logdir={model_path}/runs
```
```



If you want tensorboard support, you need to run the following commands on **Windows**:

This clears your logs:
```
rmdir /s /q runs
```

This shows your IP address:
```
powershell -command "$ipAddress = (Invoke-WebRequest -Uri 'http://ipinfo.io/ip').Content.Trim(); Write-Host 'Your IP address is: ' $ipAddress"  
```

This starts tensorboard at this address:
```
tensorboard --host 0.0.0.0 --logdir={model_path}\runs
```

In [40]:
from EarlyExitTrainer import ModelTrainer

trainer = ModelTrainer(model, device, model_dir=model_path)

In [41]:
# train the exits
#alpha_range = [i / 100 for i in range(0, 101, 2)] #[0, 0.25, 0.5, 0.55, 0.6, 0.65, 0.7, 0.725, 0.75, 0.775] + [i / 100 for i in range(80, 101, 2)]
alpha_range = [i / 100 for i in range(0, 105, 5)]

accuracies = {}
times = {}
avg_exit_idx = {}


epoch_count = 3 if model_name == "resnet" else 3
lr = 0.00000001 if model_name == "resnet" else 0.000001

for alpha in alpha_range:
    print(f"Training with alpha={alpha}")
    trainer.model = loader.load_model(num_outputs=num_classes, trained_classifiers=True) # reset the model each time
    trainer.set_alpha(alpha)
    final_accuracy, final_time, final_avg_exit_idx = trainer.train_exit_layers(train_dataloader, lr, epoch_count=epoch_count, validation_loader=test_dataloader)
    
    # write statistics
    accuracies[alpha] = final_accuracy
    times[alpha] = final_time
    avg_exit_idx[alpha] = final_avg_exit_idx


Training with alpha=0.3404
Loading EarlyExit DenseNet121 model architecture...
Adding exits...
Setting model weights...


Epoch 0 Validation Time 0.02061557262501818
Epoch 0 Validation Accuracy 0.4829621010638298


Epoch 1 Validation Time 0.028339068940345276
Epoch 1 Validation Accuracy 0.5954122340425532


Epoch 2 Validation Time 0.03378029706630301
Epoch 2 Validation Accuracy 0.8824800531914894
Training with alpha=0.3408
Loading EarlyExit DenseNet121 model architecture...
Adding exits...
Setting model weights...


Epoch 0 Validation Time 0.021169922453291874
Epoch 0 Validation Accuracy 0.48238031914893614


...

Epoch 1 Validation Time 0.015670360402858005
Epoch 1 Validation Accuracy 0.4787234042553192


Epoch 2 Validation Time 0.01562618448379192
Epoch 2 Validation Accuracy 0.4787234042553192
Training with alpha=0.36
Loading EarlyExit DenseNet121 model architecture...
Adding exits...
Setting model weights...


Epoch 0 Validation Time 0.016665309033495314
Epoch 0 Validation Accuracy 0.4787234042553192


Epoch 1 Validation Time 0.016568169948902537
Epoch 1 Validation Accuracy 0.4787234042553192


Epoch 2 Validation Time 0.016286413720313538
Epoch 2 Validation Accuracy 0.4787234042553192


In [42]:
    
# generate dataframe containing statistics
import pandas as pd

df = pd.DataFrame({
    "accuracy": accuracies,
    "time": times,
    "avg_exit_idx": avg_exit_idx
})

df

,accuracy,time,avg_exit_idx
0.3404,0.882480,0.033780,3.913398
0.3408,0.881233,0.035551,3.902926
0.3412,0.880070,0.035430,3.888215
0.3416,0.879571,0.035047,3.874751
0.3420,0.877410,0.036105,3.851562
0.3424,0.875748,0.034946,3.826380
0.3428,0.872507,0.034919,3.798953
0.3432,0.868517,0.034679,3.761802
0.3436,0.864279,0.033030,3.725150
0.3440,0.859624,0.033324,3.677776


In [43]:
# save dataframe

df.to_csv(os.path.join(model_path, "alpha_tuning.csv"), index=False)